In [2]:
import pandas as pd
import numpy as np
import string, re
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy import sparse

%matplotlib inline

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
print 'Data Loaded'

Data Loaded


In [4]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [24]:
#5109
train_df['comment_text'][(train_df.insult == 1)]

32       "\nBan one side of an argument by a bullshit n...
81       "\n\nIt was very constructive you are just ver...
86       "\n\n You know what? Fuck you! I tried to be c...
104      "\n\n Corrected spelling of ""recognised"" to ...
122      your time....\n\nYou suck because you are RIGH...
124                           YOU NEED A PUSSY, YOUR GEEK!
160      loser, get a life\n\nyou must be fat, ugly and...
213      F'UCK YOU ADMINISTRATORS (I WILL BE BACK TO TR...
292      Jimbo is bending over backwards so hard to do ...
307      Another stupid person who considers leaving a ...
308      These Iranian bastards are at it again. Not on...
330                        is the sock puppet of ,fuck off
352      Idiots\n\nThe Barris batmobile did not have a ...
436            you are a stupid ass hole thats my  message
459                                  alex fuck you \n\n...
480      AH HAAHHAHAHAH YAAA BITCH YAAAA!@!!!!!!AH HAAH...
490                              fucking loser suck a di

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
lemmatizer = nltk.stem.WordNetLemmatizer()
toRemove = ['\n', '\t', '\r']
printable = set(string.printable)
def preprocess(row):    
    try:
        comment = row['comment_text']
        comment = filter(lambda x: x in printable, comment)
        for ele in toRemove:
            comment = comment.replace(ele, '')
        comment = comment.translate(None, string.punctuation)
        #Remove user mention
        comment = re.sub('@[^\s]+','',comment)
        comment = " ".join([lemmatizer.lemmatize(word) for word in comment.split(" ")])
        
        words = comment.split(' ')
        upperCaseWords = filter(lambda x: x.isupper(), words)
        row['ProcessedText'] = comment
        row['wordCount'] = len(words)
        row['shouting words'] = len(upperCaseWords)
    except Exception as e:
        print '\n\n The comment could not be processed \n\n', row, e.message
        row["ProcessedText"] = "unknown"
        row["wordCount"] = 1
        row["shouting words"] = 0
    return row

Things that could probably help after the initial model is built
1. Check for words with all caps letter, and note the count of it and if the whole comments is in all caps
2. Make/Get a list of expletives used
3. Make features specific to each of the different attack types
    a. toxic/server_toxic - list of expletives and deragotary terms
    b. identity_hate - country / religion name or other identities like - liberal, Conservative or Gender
    c. Threat - Action by the speaker
    d. insult ?? if toxic and directed towards someone
4. use word2vec and create a KNN sort of model for prediction


The first set of models that we will try are Naive Bayes, SVM and KNN

In [37]:
combinedDf = train_df.append(test_df)
combinedDf = combinedDf.apply(preprocess, axis=1)
combinedDf.head()

,comment_text,id,identity_hate,insult,obscene,severe_toxic,threat,toxic,ProcessedText,wordCount,shouting words
0,Explanation\nWhy the edits made under my usern...,0000997932d777bf,0.0,0.0,0.0,0.0,0.0,0.0,ExplanationWhy the edits made under my usernam...,42,2
1,D'aww! He matches this background colour I'm s...,000103f0d9cfb60f,0.0,0.0,0.0,0.0,0.0,0.0,Daww He match this background colour Im seemin...,18,1
2,"Hey man, I'm really not trying to edit war. It...",000113f07ec002fd,0.0,0.0,0.0,0.0,0.0,0.0,Hey man Im really not trying to edit war Its j...,42,0
3,"""\nMore\nI can't make any real suggestions on ...",0001b41b1c6bb37e,0.0,0.0,0.0,0.0,0.0,0.0,MoreI cant make any real suggestion on improve...,112,4
4,"You, sir, are my hero. Any chance you remember...",0001d958c54c6e35,0.0,0.0,0.0,0.0,0.0,0.0,You sir are my hero Any chance you remember wh...,13,0


In [5]:
'''
def wordProcessing(row):
    words = row['ProcessedText'].split(' ')
    upperCaseWords = filter(lambda x: x.isupper(), words)
    row['wordCount'] = len(words)
    row['shouting words'] = len(upperCaseWords)
    return row
combinedDf = combinedDf.apply(wordProcessing, axis=1)
'''
combinedDf = pd.read_csv('combinedProcessedDataSet.csv')
combinedDf.head()
#combinedDf.to_csv('combinedProcessedDataSet.csv', index=False)

,comment_text,id,identity_hate,insult,obscene,severe_toxic,threat,toxic,ProcessedText,wordCount,shouting words
0,Explanation\nWhy the edits made under my usern...,0000997932d777bf,0.0,0.0,0.0,0.0,0.0,0.0,ExplanationWhy the edits made under my usernam...,42,2
1,D'aww! He matches this background colour I'm s...,000103f0d9cfb60f,0.0,0.0,0.0,0.0,0.0,0.0,Daww He match this background colour Im seemin...,18,1
2,"Hey man, I'm really not trying to edit war. It...",000113f07ec002fd,0.0,0.0,0.0,0.0,0.0,0.0,Hey man Im really not trying to edit war Its j...,42,0
3,"""\nMore\nI can't make any real suggestions on ...",0001b41b1c6bb37e,0.0,0.0,0.0,0.0,0.0,0.0,MoreI cant make any real suggestion on improve...,112,4
4,"You, sir, are my hero. Any chance you remember...",0001d958c54c6e35,0.0,0.0,0.0,0.0,0.0,0.0,You sir are my hero Any chance you remember wh...,13,0


In [5]:
label_cols = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

combinedDf['none'] = 1-combinedDf[label_cols].max(axis=1)
combinedDf = combinedDf.reset_index(drop=True)
combinedDf['percentShout'] = combinedDf['shouting words'] / (combinedDf['wordCount'] + 1)
testData = combinedDf.loc[train_df.shape[0]:].copy()
trainData, valData = train_test_split(combinedDf.loc[: train_df.shape[0] - 1].copy(), test_size = 0.1)

vec = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.9, stop_words='english',
                      strip_accents='unicode', use_idf=1,
                      smooth_idf=1, sublinear_tf=1, max_features = 800000)
trn_term_doc = pd.DataFrame(vec.fit_transform(trainData['ProcessedText']).todense())
test_term_doc = pd.DataFrame(vec.transform(testData['ProcessedText'].fillna("unknown")).todense())
val_term_doc = pd.DataFrame(vec.transform(valData['ProcessedText'].fillna("unknown")).todense())

colsCopy = ['wordCount', 'shouting words', 'percentShout']
for col in colsCopy:
    trn_term_doc[col] = trainData[col].values
    test_term_doc[col] = testData[col].values
    val_term_doc[col] = valData[col].values

In [12]:
## Defines NB SVM model
class nbsvm():
    def pr(self, y_i, y, data):
        p = data[y==y_i].sum(0)
        return (p+1) / ((y==y_i).sum()+1)

    def fit(self, data, y):
        y = y.values
        #print 'y shape', y.shape
        r = np.log(self.pr(1,y, data) / self.pr(0,y, data))
        m = LogisticRegression(C=4, dual=True)
        x_nb = np.multiply(r.reshape(1, len(r)), data.values)
        #print np.isnan(x_nb).any(), np.isnan(y).any(), np.isinf(x_nb).any(), np.isinf(y).any()
        self.model = (m.fit(x_nb, y), r)
        return self.model
    
    def predict_proba(self, data):
        m, r = self.model
        return m.predict_proba(np.multiply(r.reshape(1, len(r)), data.values))

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import log_loss, roc_auc_score
models = [('nbsvm', nbsvm()), ('extraTreeClassifier', ExtraTreesClassifier(n_jobs=-1, random_state=3))]
for mdlName, mdl in models:
    preds = np.zeros((val_term_doc.shape[0], len(label_cols)))
    print'Model Name:', mdlName
    for i, j in enumerate(label_cols):
        print 'fit:', j
        mdl.fit(trn_term_doc, trainData[j].reset_index(drop=True))
        preds[:,i] = mdl.predict_proba(val_term_doc)[:, 1]
    
    lossValue = []
    for i, col in enumerate(label_cols):
        lossValue.append(roc_auc_score(valData[col], preds[:, i]))
    print 'Loss Value', np.array(lossValue).mean()
    print '*'* 53

Model Name: nbsvm
fit: toxic


In [11]:
print val_term_doc.shape
print trn_term_doc.shape
print test_term_doc.shape

(15958, 240422)
(143613, 240422)
(153164, 240422)


## Test prediction starts here

In [ ]:
preds = np.zeros((test_term_doc.shape[0], len(label_cols)))
trn_term_doc = trn_term_doc.append(val_term_doc)
trainData = trainData.append(valData)

In [ ]:
models = [('nbsvm', nbsvm()), ('extraTreeClassifier', ExtraTreesClassifier(n_jobs=-1, random_state=3))]
for mdlName, mdl in models:
    preds = np.zeros((val_term_doc.shape[0], len(label_cols)))
    print('Model Name', mdlName)
    for i, j in enumerate(label_cols):
        print('fit', j)
        mdl.fit(trn_term_doc, trainData[j].reset_index(drop=True))
        preds[:,i] = mdl.predict_proba(test_term_doc)[:,1]
    submid = pd.DataFrame({'id': map(lambda x: str(x), testData["id"].values)})
    submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
    #print submission.shape, submission.dtypes
    submission.to_csv(mdlName + '_shoutingwords.csv', index=False)

In [15]:
models = [('nbsvm', nbsvm()), ('extraTreeClassifier', ExtraTreesClassifier(n_jobs=-1, random_state=3))]
for mdlName, mdl in models:
    preds = np.zeros((val_term_doc.shape[0], len(label_cols)))
    print'Model Name:', mdlName
    for i, j in enumerate(label_cols):
        print 'fit:', j
        mdl.fit(trn_term_doc, trainData[j].reset_index(drop=True))
        preds[:,i] = mdl.predict_proba(val_term_doc)[:, 1]
    
    lossValue = []
    for i, col in enumerate(label_cols):
        lossValue.append(roc_auc_score(valData[col], preds[:, i]))
    print 'Loss Value', np.array(lossValue).mean()
    print '*'* 53

(226998, 7)

In [45]:
submid = pd.DataFrame({'id': map(lambda x: str(x), testData["id"].values)})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
#print submission.shape, submission.dtypes
submission.to_csv('NBSVM_shoutingwords.csv', index=False)

In [46]:
submission.shape

(153164, 7)

In [47]:
submission.columns

Index([u'id', u'toxic', u'severe_toxic', u'obscene', u'threat', u'insult',
       u'identity_hate'],
      dtype='object')

In [37]:
pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1).shape

(322849, 7)

1. Create the NBSVM model
2. Create Bidirectional LSTM model
3. Create LSTM-NBSVM model
4. Use word to vector for data representation, things are from wikipedia so there should be doc2vec rep for them
5. Create a list of communities/nationality/gender etc and using entity extraction check for existence of such communities or just via text match
6. Follow the steps given here to build a better model. https://cyberbullying.org/machine-learning-can-help-us-combat-online-abuse-primer
7. Papers to read for feature engineering - https://arxiv.org/pdf/1702.06877.pdf, https://faculty.ist.psu.edu/xu/papers/Chen_etal_SocialCom_2012.pdf
8. Check for existence of third second person pronouns and close meaning such as “ur”
9. https://github.com/LDNOOBW/List-of-Dirty-Naughty-Obscene-and-Otherwise-Bad-Words/blob/master/en

The list of bad words needs to be broken down into multiple lists as they are very much different
1. Profane words
2. Sexual words
3. community words - words used to define a community
4. Racial slurs

In each of the list group the words into 3 categories [low, medium, high] which define the chance that the word can be used in a negative way

In [ ]:
secondPersonPronoun = ['you', 'ur', 'your', 'yours']
thirdPersonPronoun = ['they', 'those', 'them', 'their', 'theirs', 'these']
